In [2]:
#imports
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt


In [26]:
matchData = pd.read_csv("C:\\Users\\frank\\OneDrive\\Desktop\\T20Prediction\\Data\\MatchResults.csv")
countries = ["India", "U.S.A.", "Pakistan", "Canada", "Ireland", "Australia", "England", "Scotland", "Namibia", "Oman", "Afghanistan", 
             "West Indies", "New Zealand", "Uganda", "P.N.G.", "South Africa", "Bangladesh", "Sri Lanka", "Netherlands", "Nepal"]

matchResultsFiltered = matchData[(matchData['Team1'].isin(countries)) & (matchData['Team2'].isin(countries)) & (matchData['Country'].isin(countries))]
#matchResultsFiltered = matchData[((matchData['Team1'] == 'India') & (matchData['Team2'] == 'Australia') | (matchData['Team1'] == 'Australia') & (matchData['Team2'] == 'India'))]
#matchResultsFiltered = matchData.dropna()
len(matchResultsFiltered)

912

In [ ]:
#Collect Features
# India Win Percentage 
IndiaWinFraction = len(matchResultsFiltered[(matchResultsFiltered['Match Winner'] == 'India')])/len(matchResultsFiltered)
# India WorldCup Wins
IndiaWCWinFraction = len(matchResultsFiltered[((matchResultsFiltered['Series Name'].str.contains('ICC')) & (matchResultsFiltered['Match Winner'] == 'India'))]) / len(matchResultsFiltered[(matchResultsFiltered['Series Name'].str.contains('ICC'))])

In [29]:
X = matchResultsFiltered[['Team1', 'Team2','TossWinner', 'Country']]
y = matchResultsFiltered[['Match Winner']]

categorical_features_X = ['Team1', 'Team2', 'TossWinner', 'Country']
categorical_features_y = ['Match Winner']

encoder_X = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder_y = OneHotEncoder(sparse_output=False, handle_unknown='ignore')



X_encoded = encoder_X.fit_transform(matchResultsFiltered[categorical_features_X])
y_encoded = encoder_y.fit_transform(matchResultsFiltered[categorical_features_y])


print("Encoded X shape:", X_encoded.shape)
print("Encoded y shape:", y_encoded.shape)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.25, random_state=4)

print(encoder_X.categories_)

print(len(X_train[0]))




Encoded X shape: (912, 75)
Encoded y shape: (912, 20)
[array(['Afghanistan', 'Australia', 'Bangladesh', 'Canada', 'England',
       'India', 'Ireland', 'Namibia', 'Nepal', 'Netherlands',
       'New Zealand', 'Oman', 'P.N.G.', 'Pakistan', 'Scotland',
       'South Africa', 'Sri Lanka', 'Uganda', 'West Indies'], dtype=object), array(['Afghanistan', 'Australia', 'Bangladesh', 'Canada', 'England',
       'India', 'Ireland', 'Namibia', 'Nepal', 'Netherlands',
       'New Zealand', 'Oman', 'P.N.G.', 'Pakistan', 'Scotland',
       'South Africa', 'Sri Lanka', 'U.S.A.', 'Uganda', 'West Indies'],
      dtype=object), array(['Afghanistan', 'Australia', 'Bangladesh', 'Canada', 'England',
       'India', 'Ireland', 'Namibia', 'Nepal', 'Netherlands',
       'New Zealand', 'Oman', 'P.N.G.', 'Pakistan', 'Scotland',
       'South Africa', 'Sri Lanka', 'Uganda', 'West Indies'], dtype=object), array(['Australia', 'Bangladesh', 'Canada', 'England', 'India', 'Ireland',
       'Namibia', 'Nepal', 'Netherl

In [30]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='tanh'),
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='tanh'),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='tanh'),
    Dense(32, activation='tanh'),
    Dense(y_train.shape[1], activation='tanh') 
])

custom_optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam'
)

model.compile(optimizer=custom_optimizer,
              loss='categorical_crossentropy',  # or 'sparse_categorical_crossentropy' for multi-class classification
              metrics=['accuracy'])

In [31]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_split=0.2, verbose=0)

In [ ]:
plt.plot(history.history['accuracy'])
plt.legend(['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['loss'])
plt.legend(['loss'])
plt.title('loss')
plt.xlabel('epoch')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

In [19]:
X = [['Team1', 'Team2','TossWinner', 'Country'],
     ['Team1','mars','lars','fif']]
X = np.asarray(X)
print(X.reshape(-1,1))


encoder_X = OneHotEncoder()


X_encoded = encoder_X.fit_transform(X)



print("Encoded X shape:", X_encoded.shape)




[['Team1']
 ['Team2']
 ['TossWinner']
 ['Country']
 ['Team1']
 ['mars']
 ['lars']
 ['fif']]
Encoded X shape: (2, 7)
